In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import pennylane as qml
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import os
import cv2
from playsound import playsound

input_dim = 27
wires = qml.wires.Wires(['0', '1', '2'])
wire_map = {'0': 0, '1': 1, '2': 2}
n_layers = 2
num_qubits = 3
weight_shapes = {"weights": (n_layers, num_qubits)}
mapping_wires = wires.map(wire_map)

dev = qml.device("default.qubit", wires=wires)

@qml.qnode(dev, interface="tf")
def qnode(inputs, weights):
    qml.AngleEmbedding(features=inputs, wires=range(num_qubits), rotation='Z')
    qml.RX(weights[0], wires=0)
    qml.RX(weights[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=10)

model = Sequential([
    tf.keras.layers.Input(shape=(28, 28, 3)),
    Flatten(),
    Dense(128, activation='relu'),
    qlayer,
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

train_datagen = ImageDataGenerator(rescale=1/255, rotation_range=0.2, shear_range=0.2,
                                   zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory('C:\\HomeRepos\\M&TSI\\NNs\\data\\train',
                                               target_size=(80, 80), batch_size=8, class_mode='categorical', subset='training')
validation_data = train_datagen.flow_from_directory('C:\\HomeRepos\\M&TSI\\NNs\\data\\train',
                                                    target_size=(80, 80), batch_size=8, class_mode='categorical', subset='validation')
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory('C:\\HomeRepos\\M&TSI\\NNs\\data\\test',
                                             target_size=(80, 80), batch_size=8, class_mode='categorical')

checkpoint = ModelCheckpoint('C:/r/data/models/model2', monitor='val_loss', save_best_only=True, verbose=3)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)
callbacks = [checkpoint, earlystop, learning_rate]

history = model.fit(train_data, steps_per_epoch=train_data.samples // 32, validation_data=validation_data,
                    validation_steps=validation_data.samples // 32, callbacks=callbacks, epochs=5)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

cap = cv2.VideoCapture(0)
Score = 0

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
    
    cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0), thickness=cv2.FILLED)
    
    if len(faces) == 0 or len(eyes) == 0:
        cv2.putText(frame, 'empty', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                    thickness=1, lineType=cv2.LINE_AA)
    else:
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(255, 0, 0), thickness=3)
        
        for (ex, ey, ew, eh) in eyes:
            eye = frame[ey:ey + eh, ex:ex + ew]
            eye = cv2.resize(eye, (28, 28))
            eye = eye / 255.0
            eye = eye.reshape(1, 28, 28, 3)
            
            prediction = model.predict(eye)
            class_index = np.argmax(prediction, axis=1)
            
            if class_index == 0:
                cv2.putText(frame, 'closed', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score ' + str(Score), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                Score += 2
                if Score > 100:
                    try:
                        playsound('path_to_your_siren.mp3')
                        Score = 0
                    except:
                        pass
            
            elif class_index == 1:
                cv2.putText(frame, 'open', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score ' + str(Score), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                Score -= 5
                if Score < 0:
                    Score = 0
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(40) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'wrapt'

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import pennylane as qml
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import os
import cv2
from playsound import playsound

input_dim = 27
wires = qml.wires.Wires(['0', '1', '2'])
wire_map = {'0': 0, '1': 1, '2': 2}
n_layers = 2
num_qubits = 3
weight_shapes = {"weights": (n_layers, num_qubits)}
mapping_wires = wires.map(wire_map)

dev = qml.device("default.qubit", wires=wires)

@qml.qnode(dev, interface="tf")
def qnode(inputs, weights):
    qml.AngleEmbedding(features=inputs, wires=range(num_qubits), rotation='Z')
    qml.RX(weights[0], wires=0)
    qml.RX(weights[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=10)

model = Sequential([
    tf.keras.layers.Input(shape=(28, 28, 3)),
    Flatten(),
    Dense(128, activation='relu'),
    qlayer,
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

train_datagen = ImageDataGenerator(rescale=1/255, rotation_range=0.2, shear_range=0.2,
                                   zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory('C:\\HomeRepos\\M&TSI\\NNs\\data\\train',
                                               target_size=(80, 80), batch_size=8, class_mode='categorical', subset='training')
validation_data = train_datagen.flow_from_directory('C:\\HomeRepos\\M&TSI\\NNs\\data\\train',
                                                    target_size=(80, 80), batch_size=8, class_mode='categorical', subset='validation')
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory('C:\\HomeRepos\\M&TSI\\NNs\\data\\test',
                                             target_size=(80, 80), batch_size=8, class_mode='categorical')

checkpoint = ModelCheckpoint('C:/r/data/models/model2', monitor='val_loss', save_best_only=True, verbose=3)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)
callbacks = [checkpoint, earlystop, learning_rate]

history = model.fit(train_data, steps_per_epoch=train_data.samples // 32, validation_data=validation_data,
                    validation_steps=validation_data.samples // 32, callbacks=callbacks, epochs=5)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

cap = cv2.VideoCapture(0)
Score = 0

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
    
    cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0), thickness=cv2.FILLED)
    
    if len(faces) == 0 or len(eyes) == 0:
        cv2.putText(frame, 'empty', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                    thickness=1, lineType=cv2.LINE_AA)
    else:
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(255, 0, 0), thickness=3)
        
        for (ex, ey, ew, eh) in eyes:
            eye = frame[ey:ey + eh, ex:ex + ew]
            eye = cv2.resize(eye, (28, 28))
            eye = eye / 255.0
            eye = eye.reshape(1, 28, 28, 3)
            
            prediction = model.predict(eye)
            class_index = np.argmax(prediction, axis=1)
            
            if class_index == 0:
                cv2.putText(frame, 'closed', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score ' + str(Score), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                Score += 2
                if Score > 100:
                    try:
                        playsound('path_to_your_siren.mp3')
                        Score = 0
                    except:
                        pass
            
            elif class_index == 1:
                cv2.putText(frame, 'open', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score ' + str(Score), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                            thickness=1, lineType=cv2.LINE_AA)
                Score -= 5
                if Score < 0:
                    Score = 0
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(40) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'absl'